In [1]:
from datetime import datetime
now = datetime.now()
dt_string = now.strftime("%d%m%Y %H-%M-%S")
print("Today's date:", dt_string)

Today's date: 17042020 14-22-36


In [2]:
import pymzml
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
import glob
from tqdm import tqdm
from pathlib import Path 

In [7]:
path = 'D:/UW/massmotif/mzml/DRO_DIE_1ppm_10h_exp1_method_2.mzML'

In [8]:
scans = get_scans(path)

In [3]:
def get_scans(path, ms_all = False, ms_lv = 1):
    run = pymzml.run.Reader(path)
    scans = []
    if ms_all == False:
        for scan in run:
            if scan.ms_level == ms_lv:
                scans.append(scan)
    elif ms_all == True:
        for scan in run:
            scans.append(scan)
            
    return scans

In [4]:
def ms_qc(mzml_scans):
    # seperate ms1 and ms2
    ms1 = []
    ms2 = []
    for scan in mzml_scans:
        if scan.ms_level == 1:
            ms1.append(scan)
        elif scan.ms_level == 2:
            ms2.append(scan)
    
    # ms1 QC
    tot_ms1 = len(ms1)
    TIC = []
    for scan in ms1:
        TIC.append(scan.TIC)
        
    tic_h = max(TIC)
    tic_avg = sum(TIC) / len(TIC)
    
    #ms2 QC
    if len(ms2) != 0:
        tot_ms2 = len(ms2)
        h_response = []
        precursor_mz = []
        precursor_i = []
        for scan in ms2:
            h_response.append(scan.i.max())
            precursor_mz.append(scan.selected_precursors[0]['mz'])
            precursor_i.append(scan.selected_precursors[0]['i'])
        avg_response_h = sum(h_response) / len(h_response)
        avg_precursor_i = sum(precursor_i) / len(precursor_i)
        l_precursor_i = min(precursor_i)
        precursor_range = str(round(min(precursor_mz), 2)) + '~' + str(round(max(precursor_mz),2))
    
    result = [tot_ms1, "{:.2e}".format(tic_h), "{:.2e}".format(tic_avg), tot_ms2, round(avg_response_h, 2), "{:.2e}".format(avg_precursor_i), round(l_precursor_i, 2), precursor_range]

    return result

In [5]:
def batch_scans(path):
    all_files = glob.glob(path + "/*.mzML")
    scans = []
    file_list = []
    for file in tqdm(all_files):
        scan = get_scans(file, True)
        scans.append(scan)
        file_list.append(Path(file).name)
    print(file_list)
    print('Batch read finished!')
    
    return scans, file_list

In [6]:
def qc_gen(path):
    batch_scan, file_list = batch_scans(path)
    print('All files read in!')
    
    qc_result = []
    ran_scan = len(batch_scan)
    print('Generating QC report...')
    for index in tqdm(np.arange(ran_scan)):
        result = ms_qc(batch_scan[index])
        
        result = [file_list[index]] + result
        qc_result.append(result)
    print('Generating dataframe...')
    col = ['file_name', 'total_ms1_scan', 'max_tic', 'avg_tic','tot_ms2', 'avg_ms2_max', 'avg_ms2precursor_i', 'min_ms2precursor_i', 'ms2precursor_range']
    d_result = pd.DataFrame(qc_result, columns = col)
    print('Finished!')
    
    return d_result

In [10]:
qc_gen('../mzml/20200416mzml/')

100%|██████████| 6/6 [00:14<00:00,  2.47s/it]


['20200413-DRODIE 1ppm 10h exp1 autoMSMS neg.mzML', '20200413-DRODIE 1ppm 10h exp2 autoMSMS neg.mzML', '20200413-DRODIE 1ppm 29h exp1 autoMSMS neg.mzML', '20200413-DRODIE 1ppm 29h exp2 autoMSMS neg.mzML', '20200413-DRODIE 1ppm 4h exp1 autoMSMS neg.mzML', '20200413-DRODIE 1ppm 4h exp2 autoMSMS neg.mzML']
Batch read finished!
All files read in!
Generating QC report...


100%|██████████| 6/6 [00:05<00:00,  1.17it/s]


Generating dataframe...
Finished!


,file_name,total_ms1_scan,max_tic,avg_tic,tot_ms2,avg_ms2_max,avg_ms2precursor_i,min_ms2precursor_i,ms2precursor_range
0,20200413-DRODIE 1ppm 10h exp1 autoMSMS neg.mzML,3000,4.41e+06,1.99e+05,1458,3729.09,6.85e+04,9654.36,200.01~499.63
1,20200413-DRODIE 1ppm 10h exp2 autoMSMS neg.mzML,2915,4.52e+06,2.09e+05,1551,3650.83,6.86e+04,9101.39,200.17~499.63
2,20200413-DRODIE 1ppm 29h exp1 autoMSMS neg.mzML,2884,4.57e+06,2.06e+05,1572,3496.00,6.82e+04,8048.62,200.01~499.62
3,20200413-DRODIE 1ppm 29h exp2 autoMSMS neg.mzML,2898,4.63e+06,2.10e+05,1566,3745.36,6.96e+04,13063.84,201.01~499.63
4,20200413-DRODIE 1ppm 4h exp1 autoMSMS neg.mzML,2930,4.56e+06,2.06e+05,1548,3558.53,6.85e+04,9287.11,200.01~499.62
5,20200413-DRODIE 1ppm 4h exp2 autoMSMS neg.mzML,2941,4.57e+06,2.08e+05,1548,3802.99,6.83e+04,9802.40,200.01~499.63


# Target Feature Top N tracer

In [33]:
def mz_locator(input_list, mz, error, select_app = True): #updated to select_app, when false only select closest one, when true append all, use as a backdoor for now if closest algorithm messed up
    '''
    Find specific mzs from given mz and error range out from a given mz array
    input list: mz list
    mz: input_mz that want to be found
    error: error range is now changed to ppm level
    '''
    target_mz = []
    target_index = []
    
    #ppm conversion
    error = error * 1e-6
    
    lower_mz = mz - error * mz
    higher_mz = mz + error * mz

    for i, mzs in enumerate(input_list):
        if mzs < lower_mz:
            continue
        elif mzs >= lower_mz:
            if mzs <= higher_mz:
                target_mz.append(mzs)
                target_index.append(i)

    if select_app == False:
        if len(target_mz) != 0:
            target_error = [abs(i - mz) for i in target_mz]
            minpos = target_error.index(min(target_error)) 
            t_mz = target_mz[minpos]
            t_i = target_index[minpos]
        else:
            t_mz = 0
            t_i = 'NA'
    if select_app == True:
        t_mz = target_mz
        t_i = target_index
        
    return t_mz, t_i

In [73]:
def topn_search(mzml_scans, mz, error):
    result = []
    for scan in tqdm(mzml_scans):
        mz_i_pair = list(zip(scan.mz, scan.i))
        mz_i_pair.sort(key=lambda x: x[1], reverse=True)
        def unzip(iterable):
            return list(zip(*iterable))
        sorted_mz = unzip(mz_i_pair)[0]
        sorted_i = unzip(mz_i_pair)[1]
        t_mz, t_i = mz_locator(sorted_mz, mz, error, False)
        if t_i != 'NA':
            result.append([mz_i_pair[t_i], t_i, scan.scan_time[0]])
    
    return result

In [72]:
r_test = topn_search(scans, 255.1576, 50)

In [155]:
def topn_summary_s(mzml_scans, mz, error, rt_min = 0, rt_max = 30, topN = 5):
    
    r_topn = topn_search(mzml_scans, mz, error)
    r_topn = [i for i in r_topn if i[2] >= rt_min and i[2] <=rt_max]
    
    def get_col(arr, col):
        return list(map(lambda x : x[col], arr))
    
    mz_list = get_col(get_col(r_topn, 0), 0)
    i_list = get_col(get_col(r_topn, 0), 1)
    top_list = get_col(r_topn, 1)
    
    r_top10 = [i for i in r_topn if i[1] <= (topN - 1)]
    i_list_t10 = get_col(get_col(r_top10, 0), 1)
    
    if len(r_top10) > 0:
        i_range = str(round(min(i_list_t10), 2)) + '~' + str(round(max(i_list_t10), 2))

        n_range = str(int(max(top_list)+1)) + '~' + str(int(min(top_list)+1))

        print('intensity range when in top', topN, ':', i_range, ', topn_range', n_range, ', total scans in top', topN, ':', len(i_list_t10))

        number_topn = get_col(r_top10, 1)
        plt.figure(figsize = (10,6))
        plt.hist(number_topn, bins = topN);
        plt.xlim(0,topN)
        plt.show()
    else:
        print('not found within selected top N range!')
    
    return i_list, top_list

In [160]:
i_list, top_list = topn_summary_s(scans, 225.1576, 50, 10)

100%|██████████| 945/945 [00:24<00:00, 38.68it/s] 


not found within selected top N range!


In [171]:
def topn_summary_b(mzml_scans, mzs, error, rt_min = 0, rt_max = 30, topN = 5):
    
    def get_col(arr, col):
        return list(map(lambda x : x[col], arr))
    result = []
    for mz in mzs:
        r_topn = topn_search(mzml_scans, mz, error)
        r_topn = [i for i in r_topn if i[2] >= rt_min and i[2] <=rt_max]

        mz_list = get_col(get_col(r_topn, 0), 0)
        i_list = get_col(get_col(r_topn, 0), 1)
        top_list = get_col(r_topn, 1)
        
        
        r_top10 = [i for i in r_topn if i[1] <= (topN - 1)]
        if len(r_top10) > 0:
            i_list_t10 = get_col(get_col(r_top10, 0), 1)

            i_range = str(round(min(i_list_t10), 2)) + '~' + str(round(max(i_list_t10), 2))

            n_range = str(int(max(top_list)+1)) + '~' + str(int(min(top_list)+1))

            result.append([mz, i_range, n_range, len(i_list_t10)])
            d_result = pd.DataFrame(result, columns = ['mz', 'intensity range in topN', 'ranking range', 'total scan in selected topN'])
        else:
            print('not found within selected top N range!')
            continue
        
    return d_result

In [169]:
result = topn_summary_b(scans, [225.1576, 245.1617], 50)

100%|██████████| 945/945 [00:27<00:00, 34.34it/s] 


In [170]:
result

,mz,intensity range,ranking range,total scan in selected topN
0,225.1576,87877.27~1148033.6,553~1,337
1,245.1617,57825.55~1147859.1,2543~1,400
